In [ ]:
from helper_func import load_images 
load_images()

## *CONVERT TO SQL CELL FROM MARKDOWN* & Delete this line

-- Create internal stage for images with server-side encryption
CREATE OR REPLACE STAGE IMAGE_STAGE
    DIRECTORY = (ENABLE = TRUE)
    ENCRYPTION = (TYPE = 'SNOWFLAKE_SSE');

In [ ]:
CREATE OR REPLACE TABLE IMAGE_CATALOG AS
SELECT 
   TO_FILE(FILE_URL) AS IMG_FILE, 
   * 
FROM DIRECTORY(@IMAGE_STAGE);

# AI SQL

In [ ]:
select 
img_file,
relative_path,
snowflake.cortex.complete(
    'pixtral-large', 
    prompt('What flight id in this image: {0}."', img_file)
) AS AI_COMPLETE_RESULT,
from IMAGE_CATALOG

In [ ]:
select 
img_file,
relative_path,
snowflake.cortex.complete(
    'pixtral-large', 
    prompt('What flight id in this image: {0}. Provide the response in this exact format: "flight_id"', img_file)
) AS AI_COMPLETE_RESULT,
from IMAGE_CATALOG

In [ ]:
select 
img_file,
relative_path,
AI_EXTRACT(
    file => img_file,
    responseFormat => [
        ['feature1', 'What flight number in this image: {0}']
    ]
):response.feature1 AS AI_EXTRACT_RESULT
from IMAGE_CATALOG

In [ ]:
select 
img_file,
relative_path
from IMAGE_CATALOG
where AI_FILTER(prompt('The boarding pass is from Jetstar {0}', img_file))

In [ ]:
SELECT
  AI_SENTIMENT(
    REVIEW_TEXT
  ),
  REVIEW_TEXT
  FROM S3_GOLD.CUSTOMER_REVIEWS LIMIT 10;

# Doc AI

In [ ]:
SELECT 
    img_file,
    relative_path,
    RETAIL_STREAMING_DEMO.PUBLIC.BOARDING_PASS!PREDICT(GET_PRESIGNED_URL(@IMAGE_STAGE, RELATIVE_PATH), 2):FLIGHT_ID AS DOC_AI_RESULT
FROM IMAGE_CATALOG